# Transformers

```{note}
Transformers 是由 Hugging Face 开发的一个开源 Python 库，目前被认为是自然语言处理乃至整个深度学习领域的事实标准工具。它的核心理念是让所有人都能轻松使用最先进（SOTA）的模型。它有三大核心组件：
1. Tokenizer： 负责数据预处理。它将人类读得懂的文本转换成模型读得懂的数字（Input IDs）。
2. Model： 负责处理数据。它是神经网络的本体（例如 BERT 或 GPT-2 的架构），负责接收数字输入并输出结果。
3. Config： 负责管理模型的架构参数（如层数、隐藏单元大小等），确保模型结构正确。
```

## 使用 Transformers

### 联网使用

以 Qwen3-0.6B 为例进行说明。

1. 加载 tokenizer 和 model
    * 前提条件是能联上 hugging face
    * 下面的代码如果第一次运行，会下载模型到本地，默认会下载到 `~/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B`
    * 后续运行可能还是需要能连 hugging face，比如有机制尝试联网验证是否为 Mistral 模型。（后面会讲如果联不上该怎么办）
    * `torch_dtype="auto"` 告诉加载器“ 请使用模型原始训练时的精度 ”来加载权重。Qwen3-0.6B 在 `config.json` 里记录了它是用 bfloat16 训练的，所以 auto 会自动选择 bfloat16 。
    * `device_map="auto"` 自动规划模型的每一层应该放在哪里（GPU 还是 CPU）。优先 GPU，如果模型太大它会自动把放不下的层切分到 CPU 内存 中。


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


2. 准备输入数据
    * `tokenize=False` 使得 text 是字符串而不是 token ids
    * `add_generation_prompt`
        * False: 返回 '<|im_start|>user\nWho are you?<|im_end|>\n'
        * True: 返回 '<|im_start|>user\nWho are you?<|im_end|>\n<|im_start|>assistant\n'
    * `return_tensors="pt"`: 返回 PyTorch 张量

In [13]:
# prepare the model input
prompt = "Who are you?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
text

'<|im_start|>user\nWho are you?<|im_end|>\n<|im_start|>assistant\n'

In [14]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
model_inputs

{'input_ids': tensor([[151644,    872,    198,  15191,    525,    498,     30, 151645,    198,
         151644,  77091,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

3. 文本补全
    * `max_new_tokens=2048`: 即 output_ids 的最大长度
    * `skip_special_tokens=True`: 跳过 `<|im_end|>` 等特殊 token，不包括 `<think>` 和 `</think>`


In [19]:
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048  # 演示用 2048 即可
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
output_ids[: 10]

[151667, 198, 32313, 11, 279, 1196, 4588, 11, 330, 15191]

In [20]:
content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
print("content:", content)

content: <think>
Okay, the user asked, "Who are you?" I need to respond appropriately. First, I should acknowledge their question. I can say something like, "Hi there! I'm an AI assistant." Then, I should explain what I can do. Maybe mention helping with tasks like writing, solving problems, or providing information. It's important to keep the response friendly and open-ended so the user feels comfortable asking more questions. I should make sure the tone is helpful and not too technical. Let me put that together.
</think>

Hi! I'm an AI assistant designed to help with a wide range of tasks, from writing and problem-solving to providing information. How can I assist you today? 😊


### 未联网使用

1. 之前已经 load 过 tokenizer 和 model，现在使用本地绝对路径进行加载
    * `os.path.expanduser` 把路径里的波浪号 ~ 翻译成 实际的用户主目录路径 
    * `local_files_only=True` 命令 transformers 库只在本地找文件，不联网找

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# 使用本地绝对路径，避免联网检查触发 ConnectError
model_name = os.path.expanduser("~/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B/snapshots/c1899de289a04d12100db370d81485cdf75e47ca")
model_name

'/home/newfacade/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B/snapshots/c1899de289a04d12100db370d81485cdf75e47ca'

In [2]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    local_files_only=True
)

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


2. 假设之前没有 load 过，要想方法下载下来相关文件
    * 一种方法是 https://huggingface.co/Qwen/Qwen3-0.6B/tree/main 中逐个下载文件，再转移到开发的目录
    * 更好的方法是 clone 整个 repo，直接 clone（clone 也需要联网，这里仅进行展示，它还在 .git 目录存储了所有文件的压缩副本，占用空间几乎翻倍）：
        ```bash
        sudo apt update
        sudo apt install git-lfs
        git lfs install
        git clone https://huggingface.co/Qwen/Qwen3-0.6B
        ```

In [5]:
# 假设下载或 clone 下来的文件都放在此目录下
model_name = '/home/newfacade/projects/Qwen3-0.6B'
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    local_files_only=True
)

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


## 文件结构

典型的 transformers 模型目录结构如下：

```bash
(base) newfacade@DESKTOP-R72H6BL:~/projects/Qwen3-0.6B$ du -sh *
12K     LICENSE
16K     README.md
4.0K    config.json
4.0K    generation_config.json
1.6M    merges.txt
1.5G    model.safetensors
11M     tokenizer.json
12K     tokenizer_config.json
2.7M    vocab.json
```

1. 模型权重 model.safetensors
2. 模型配置
    * config.json：模型的“说明书” 。它告诉代码这是一个 `Qwen3ForCausalLM` 架构的模型，有 28 层等等。
    * generation_config.json：生成时的默认参数，比如 temperature，EOS token ID 是多少等。
3. Tokenizer 相关，这几个文件共同决定了如何把“文本”变成“数字 ID”
    * tokenizer.json：
    * vocab.json：词表映射，单纯的 token string -> ID 的字典。
    * merges.txt：BPE 合并规则，记录了哪些字符对应该合并（比如 e + s -> es ）。
    * tokenizer_config.json：分词器的配置 ，比如：用哪个类来加载（ Qwen2Tokenizer ），特殊的 Token 映射。
4. 其他文件
    * LICENSE
    * README.md